In [1]:
import pandas as pd # Do not change these imports
import numpy as np
import math

In [2]:
def aggregate(path):
    df = pd.read_csv(path) # Do not change this line of code
    
    # Drop Unneccesary columns
    df.drop(['PERMNO', 'COMNAM'], axis=1, inplace = True)
    
    # Replace all NaN values with 0
    df = df.fillna(0)
    
    # Negative is used to represent something else in the data, so abs will remove the negative
    df['PRC'] = abs(df['PRC'])
    
    # Filter out rows where price is zero or DIVAMT is zero
    df = df.loc[df["PRC"] != 0]
    df = df.loc[df["DIVAMT"] != 0]
    
    # Replace date
    df["date"] = df["date"].str.slice(stop = 4)
    
    groups = df.groupby(["CUSIP", "date"], group_keys = False)
    

    price =  groups["PRC"].agg("last")
    divs = list(groups["DIVAMT"].sum())
    cusips = list(price.index.get_level_values(0))
    year = list(price.index.get_level_values(1))
    price = list(price)
    
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips,
        "Year" : year,
        "Dividends" : divs,
        "Price" : price
    })
    
    
    # filter singletons
    groups = output.groupby(["CUSIP"], group_keys = False)
    filtered = groups.filter(lambda x: len(x) != 1).reset_index().drop(['index'], axis = 1)
    
    return filtered

In [3]:
def overvalued(annual_data):
    '''
    Returns the percentage of securities that are 
    overvalued by the Dividend Discount Model. 
    The equity cost of capital is defined to be 
    (Div1 + P1 - P0) / P0, where P0 is the price of 1 share
    of the stock today, P1 is the price of the stock in
    one year, Div is the dividends per share paid 
    over the year. 
    
    TO Evaluate the P0, we use the constant growth model,
    that is P0 = (Div(n+1))/ (RE - g), where RE means the 
    equity cost of capital, and g represents the growth rate.
    
    overvalued: DataFrame -> Float 
    '''
    annual = annual_data.copy()
    
    annual = annual.sort_values(by=["CUSIP", "Year"]) 
    
    # 1. Calculate annual dividend growth.
    annual["DivGrowth"] = annual.groupby("CUSIP")["Dividends"].pct_change()
    
    # 2. Calculate the equity cost of capital
    annual['P1'] = annual.groupby("CUSIP")["Price"].shift(-1)
    annual['Div1'] = annual.groupby("CUSIP")["Dividends"].shift(-1)
    annual['re'] = (annual['Div1'] + annual['P1'] - annual['Price']) / annual['Price']
    
    # Dropping the NaNs after calculation, as required. 
    annual_divgrowth = annual.dropna(subset=["DivGrowth"])
    annual_re = annual.dropna(subset=["re"])
    
    # Compute average dividend growth per CUSIP
    # Compute average equity cost of capital
    g = annual_divgrowth.groupby("CUSIP")["DivGrowth"].mean().rename("g")
    avg_cost_cap = annual_re.groupby("CUSIP")['re'].mean().rename("rE")
    
    #3. Calculate the price on the security's last available date
    last_data = annual.groupby("CUSIP").tail(1).set_index("CUSIP")
    Dn = last_data["Dividends"].rename("Dn")
    Pn = last_data["Price"].rename("Pn")
    
    # Merging the data into ONE dataframe
    data = pd.concat([Dn, Pn, g, avg_cost_cap], axis=1, join='inner')
    data['Dn+1'] = data['Dn'] * (1 + data['g'])
    
    # Calculate the price of which g >= rE
    data['rE - g'] = data['rE'] - data['g']
    
    # Calculating the P0 based on the DDM (Constant Model)
    data['P0'] = data['Dn+1'] / (data['rE'] - data['g'])
    
    # For Values that rE - g <= 0, set P0 to infinity. 
    data.loc[(data['P0'] <= 0) | (data['rE'] - data['g'] <= 0), 'P0'] = np.inf
    
    data['Overvalued'] = data['Pn'] > data ['P0']
    
    overval_percent = (data['Overvalued'].mean()) * 100
    
    return overval_percent


overval_percent = overvalued(aggregate("monthlycrsp.csv"))
print("Percentage of overvalued securities: {}%".format(overval_percent))


Percentage of overvalued securities: 36.512%
